In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import pickle
from pprint import pprint

In [2]:
idf_df = pd.read_csv('transac_idf_cleaned.csv')
# db cleaned is without all real estate > 30000€/m2

In [3]:
# define based on real estate basic datas X => as longitude and latitude for a price per square meter /!\ carefull with X order
X = idf_df[['longitude', 'latitude']].values
y = idf_df['prix_m2'].values

In [4]:
#after running gridsearch => define the best model
best_model = RandomForestRegressor(max_depth=25)
best_model.fit(X, y)

RandomForestRegressor(max_depth=25)

In [5]:
pickle.dump(best_model, open('RFR_2X_best_model_V2.pkl', 'wb'))

In [6]:
reloaded_model = pickle.load(open('RFR_2X_best_model_V2.pkl', 'rb'))

In [7]:
reloaded_model.predict([[2.39, 43.5]])

array([2489.78952188])

In [8]:
one_sample = idf_df.sample()[['prix_m2', 'adresse', 'latitude', 'longitude']]
pprint(one_sample)
print(reloaded_model.predict(one_sample[['longitude', 'latitude']].values))

           prix_m2         adresse   latitude  longitude
24175  9666.666667  40 RUE DE TOUL  48.840623   2.403259
[9603.19319677]


In [9]:
one_sample = idf_df.sample()[['prix_m2', 'adresse', 'latitude', 'longitude']]
pprint(one_sample)
print(reloaded_model.predict(one_sample[['longitude', 'latitude']].values))

           prix_m2     adresse   latitude  longitude
86709  2515.151515  5 RUE NOEL  48.952498    2.32849
[2962.02087704]


In [10]:
import uvicorn
from fastapi import FastAPI, HTTPException


ImportError: cannot import name 'Doc' from 'typing_extensions' (c:\Users\Utilisateur\anaconda3\Lib\site-packages\typing_extensions.py)

In [ ]:
app = FastAPI(title='ESTIMATION')

#model = pickle.load(open('/mnt/c/Users/Utilisateur/Desktop/project_briefs/003_real_estate_api_estimate/RFR_2X_best_model_V2.pkl', 'rb'))
#model = pickle.load(open('RFR_2X_best_model_V2.pkl', 'rb'))
#model = pickle.load(r'C:\Users\Utilisateur\Desktop\project_briefs\003_real_estate_api_estimate\RFR_2X_best_model.pkl')
#model = pickle.load(open('RFR_2X_best_model.pkl', 'rb'))

@app.post("/price_predictor_v1/", description="With a given longitude and latitude, returns price per square meter best estimate")
async def price_predictor(longitude: float, latitude: float):
    input_data = np.array([[longitude, latitude]])
    return reloaded_model.predict(input_data)[0]


#uvicorn.run(app)
# function if main is not in the name of uvicorn app
if __name__ == '__main__':
    uvicorn.run(app, host='localhost', port=5000)



RuntimeError: asyncio.run() cannot be called from a running event loop